In [3]:
master <- read.table("../../../../../data/initial_dataset/Master_04_10_2019.csv",sep= ",",header=T)

In [4]:
write.table(master,"../../../../../data/initial_dataset/Master_04_10_2019.tsv")

In [ ]:
k <- re